In [1]:
# This notebook applies the different denoising techniques to a set of 15 grayscale images to evaluate average performance
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.filters import gaussian
from scipy.ndimage import median_filter
from skimage.metrics import peak_signal_noise_ratio as compute_psnr
from skimage.metrics import structural_similarity as compute_ssim

from filters_hw2 import *
from wnnm_final import *

import pandas as pd
# Initialize data frames to store computed psnr and ssim values
psnr_data = pd.DataFrame()
ssim_data = pd.DataFrame()

In [2]:
# Specify image names, number of images, and initialize lists to store the output
img_names = ['2','7','12','13','15','16','20','22',
             '23','25','27','28','38','47','48']
psnr_data['Image names'] = pd.DataFrame(img_names)
N_img = len(img_names)
clean_images = []
noisy_images = []
noisy_psnr = []
noisy_ssim = []
# Specify the noise to be added
sigma_noise = 0.1

# Load the images, compute psnr of the noisy images, and save the actual images being denoised
for i in range(N_img):
    clean_images.append(io.imread('test_images_greyscale/'+img_names[i]+'.gif').astype(float)/255)
    noisy_images.append(np.clip(clean_images[-1] + np.random.normal(loc=0.0, scale=sigma_noise, size=clean_images[-1].shape),0,1))
    psnr = compute_psnr(clean_images[-1],noisy_images[-1])
    noisy_psnr.append(psnr)
    ssim = compute_ssim(clean_images[-1],noisy_images[-1])
    noisy_ssim.append(ssim)
    
    io.imsave('results_greyscale/clean/'+img_names[i]+'.png', (clean_images[-1]*255).astype(np.uint8))
    io.imsave('results_greyscale/noisy/'+img_names[i]+'.png', (noisy_images[-1]*255).astype(np.uint8))

# Print and record the metrics
print('Average PSNR of noisy images = '+str(np.mean(noisy_psnr)))
psnr_data['noisy'] = noisy_psnr
print('Average SSIM of noisy images = '+str(np.mean(noisy_ssim)))
ssim_data['noisy'] = noisy_ssim

Average PSNR of noisy images = 20.22248462055455
Average SSIM of noisy images = 0.48826909867503676


In [3]:
# Perform gaussian filtering
gaus_imgs = []
gaus_psnrs = []
gaus_ssims = []

sigma = 0.9
for i in range(N_img):
    filtered = gaussian(noisy_images[i],sigma)
    psnr = compute_psnr(clean_images[i],filtered)
    ssim = compute_ssim(clean_images[i],filtered)
    
    gaus_imgs.append(filtered)
    gaus_psnrs.append(psnr)
    gaus_ssims.append(ssim)
    io.imsave('results_greyscale/gaussian/'+img_names[i]+'.png', (gaus_imgs[-1]*255).astype(np.uint8))
    
print('Average PSNR gaussian = '+str(np.mean(gaus_psnrs)))
psnr_data['gaussian'] = gaus_psnrs
print('Average SSIM gaussian = '+str(np.mean(gaus_ssims)))
ssim_data['gaussian'] = gaus_ssims

Average PSNR gaussian = 26.35892405682172
Average SSIM gaussian = 0.7979498239343151


In [5]:
# Perform bilateral filtering
bilateral_imgs = []
bilateral_psnrs = []
bilateral_ssims = []

sigmaIntensity = 0.4
for i in range(N_img):
    filtered = bilateral2d(noisy_images[i],radius=filtSize,
                                        sigma=sigma,sigmaIntensity=sigmaIntensity)
    psnr = compute_psnr(clean_images[i],filtered)
    ssim = compute_ssim(clean_images[i],filtered)
    
    bilateral_imgs.append(filtered)
    bilateral_psnrs.append(psnr)
    bilateral_ssims.append(ssim)
    io.imsave('results_greyscale/bilateral/'+img_names[i]+'.png', (bilateral_imgs[-1]*255).astype(np.uint8))
    
print('Average PSNR bilateral = '+str(np.mean(bilateral_psnrs)))
psnr_data['bilateral'] = bilateral_psnrs
print('Average SSIM bilateral = '+str(np.mean(bilateral_ssims)))
ssim_data['bilateral'] = bilateral_ssims

Average PSNR bilateral = 26.606014372622116
Average SSIM bilateral = 0.7984888042919404


In [6]:
# Perform nlm filtering
nlm_imgs = []
nlm_psnrs = []
nlm_ssims = []

nlmSigma = 0.1
averageFilterRadius = 3
searchWindowRadius = 9
for i in range(N_img):
    filtered = nonlocalmeans(noisy_images[i].reshape((noisy_images[i].shape[0],noisy_images[i].shape[1],1)),searchWindowRadius,
                             averageFilterRadius,sigma,nlmSigma).reshape((noisy_images[i].shape[0],noisy_images[i].shape[1]))
    psnr = compute_psnr(clean_images[i],filtered)
    ssim = compute_ssim(clean_images[i],filtered)
    
    nlm_imgs.append(filtered)
    nlm_psnrs.append(psnr)
    nlm_ssims.append(ssim)
    io.imsave('results_greyscale/nlm/'+img_names[i]+'.png', (nlm_imgs[-1]*255).astype(np.uint8))
    
print('Average PSNR non local means = '+str(np.mean(nlm_psnrs)))
psnr_data['nlm'] = nlm_psnrs
print('Average SSIM non local means = '+str(np.mean(nlm_ssims)))
ssim_data['nlm'] = nlm_ssims

Average PSNR non local means = 26.972477045248898
Average SSIM non local means = 0.8227874707085767


In [ ]:
# Perform WNNM filtering
wnnm_imgs = []
wnnm_psnrs = []
wnnm_ssims = []

patchRadius = 3
delta = 0.1
c = 0.005
K = 3
sigma_n = 0.1
N_threshold = 50
for i in range(N_img):
    filtered = wnnm(noisy_images[i].reshape((noisy_images[i].shape[0],noisy_images[i].shape[1],1)),patchRadius, delta, c, K,
                            sigma_n,N_threshold).reshape((noisy_images[i].shape[0],noisy_images[i].shape[1]))
    psnr = compute_psnr(clean_images[i],filtered)
    ssim = compute_ssim(clean_images[i],filtered)
    
    wnnm_imgs.append(filtered)
    wnnm_psnrs.append(psnr)
    wnnm_ssims.append(ssim)
    io.imsave('results_greyscale/wnnm/'+img_names[i]+'.png', (wnnm_imgs[-1]*255).astype(np.uint8))
    
print('Average PSNR WNNM = '+str(np.mean(wnnm_psnrs)))
psnr_data['wnnm'] = wnnm_psnrs
print('Average SSIM WNNM = '+str(np.mean(wnnm_ssims)))
ssim_data['wnnm'] = wnnm_ssims

In [ ]:
# Store computed psnr and ssim values in excel spreadsheets
psnr_data.to_excel('results_greyscale/psnrs.xlsx')
ssim_data.to_excel('results_greyscale/ssims.xlsx')